In [ ]:
!pip install jupyter-dash

In [9]:
# run dash in jupyter
from jupyter_dash import JupyterDash # thay cho: from dash import Dash
from dash import Dash, html, dcc, Input, Output, State
import pandas as pd
import plotly.express as px

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# data
happiness = pd.read_csv('/content/drive/MyDrive/GitCode/My_learning/23. Visualization/1. Plotly/data/13 - worldhappiness.csv')
happiness

,country,region,happiness_rank,happiness_score,year
0,Australia,Australia and New Zealand,10,7.284,2015
1,Australia,Australia and New Zealand,9,7.313,2016
2,Australia,Australia and New Zealand,10,7.284,2017
3,Australia,Australia and New Zealand,10,7.272,2018
4,Australia,Australia and New Zealand,11,7.228,2019
...,...,...,...,...,...
1083,Botswana,Sub-Saharan Africa,142,3.471,2022
1084,Rwanda,Sub-Saharan Africa,143,3.268,2022
1085,Zimbabwe,Sub-Saharan Africa,144,2.995,2022
1086,Lebanon,Middle East and Northern Africa,145,2.955,2022


## Basis

In [12]:
line_fg = px.line(                                                  # add line plot
    happiness[happiness['country'] == 'United States'],
    x = 'year',
    y = 'happiness_score',
    title='Happiness Score in United States'
)

# html Component:
#     section headings {H1, H2, ...}
#     paragraph {p} (text noi dung), 
#     content division {Div} ( vung chua noi dung), 
#     line-break {br}, 
#     anchor{a} ( gan hyperlink), 
#     html tags 

# app = Dash()
app = JupyterDash()

app.layout = html.Div(children=[
    html.H1('The Sample Dashboard'),                                # header 1
    html.Div(                                                       # add subdiv
        children='123',                                                 # component of subdiv
        id='output-text'                                                # set id for component
        ),
                       
    html.P([                                                        # paragraph
        'This is the sample dashboard display the scores',              # text
        html.Br(),                                                      # line break
        html.A(                                                         # hyperlink
            'World happiness Report Data Source',                           # display
            href='http://www.worldhappiness.report',                        # link
            target= '_blank'                                                # open new tab when clicked link
            ),

    dcc.RadioItems(                                                 # select ONE item in current
        options = happiness['region'].unique() ,                        # list/dict of option values can be choose
        value= 'North America'                                          # first item selected when open
        ),
    dcc.Checklist(                                                 # select MULTI-items in current
        options = happiness['region'].unique() ,                        # list/dict of option values can be choose
        value= ['North America']                                        # first items selected when open
        ),
    dcc.Dropdown(                                                  # select from dropdown list items   
        options = happiness['country'].unique() ,                       # list/dict of option values can be choose
        value= 'United States'                                     
        ),
        
    html.Br(),                                                      # line break
    dcc.Input(                                                     # add Input box component
        value='Texttttt', 
        type='text'
        ),
        
    dcc.Graph(                                                      # add graph component 
        figure =  line_fg                                          
        ),
    ])
])

# app.run_server(mode='external', host='localhost',port=8050, debug = True) # set debug = True cho refresh server with update code
# app.run_server(mode='external', debug = True)

TypeError: ignored

## Callback
 refresh dashboard after change component

In [ ]:
# callback single input/output with id

from dash import Dash, html, dcc, Input, Output

app = JupyterDash()

app.layout = html.Div([
    dcc.Input(id='input-text', value='Change this text', type='text'),
    html.Div(children='123', id='output-text')
                       ])

# Output must be before Input
@app.callback(
    Output(
        component_id='output-text',  # id of component
        component_property='children' # chỉ định tham số thay đổi value của component
        ),
    Input(
        component_id='input-text', 
        component_property='value'
        )
    )
def update_output_div(input_text):
    return f'Text: {input_text}'

app.run_server(mode='inline', host='localhost',port=8050, debug = True)

In [ ]:
# callback single input/output without id setting

app = JupyterDash()

input_text = dcc.Input(value='Input the description', type='text')
output_text = html.Div(children='123', id='output-text')

app.layout = html.Div([input_text, output_text])

@app.callback(
    Output(component_id=output_text, component_property='children'),
    Input(component_id=input_text, component_property='value')
)
def update_output_div(input_text):
    return f'Text: {input_text}'

app.run_server(mode='inline', host='localhost',port=8050, debug = True)

In [ ]:
happiness.head(2)

In [ ]:
# interactive graph


object_name = {'happiness_score':'Happiness Score','happiness_rank':'Happiness Rank'}
header = html.H1(children = 'The Sample Dashboard')
line_break = html.Br()
region_checklist = dcc.Checklist(options = happiness['region'].unique() , value= ['North America'])
country_dropdown = dcc.Dropdown(options = happiness['country'].unique() , value= 'United States')
object_ratio = dcc.RadioItems(options = object_name , value= 'happiness_score') # dict in options if "the interface value" is different from "the data value"
line_graph = dcc.Graph(figure =  line_fg)
avg_cal_update_button = html.Button(n_clicks = 0, children = 'Update the average happiness Score/Rank:')
avg_cal_info = html.Div()

app = JupyterDash()
app.layout = html.Div(children=[header, line_break, region_checklist, country_dropdown, line_break, object_ratio, line_graph, avg_cal_update_button, avg_cal_info])

# chain callback: output callback1 is input callback2 (nested filter/slicer)
@app.callback(Output(country_dropdown,'options'), 
              Output(country_dropdown,'value'), 
              Input(region_checklist,'value'))
def update_country(selected_regions):
    new_country_dropdown = happiness[happiness['region'].isin(selected_regions)]['country'].unique()
    return new_country_dropdown , new_country_dropdown[0]

@app.callback(Output(line_graph,'figure'), 
              Input(object_ratio,'value'),
              Input(region_checklist,'value'),
              Input(country_dropdown,'value')) # số input và output của callback trùng với số input và output của hàm def
def update_graph(graph_object,selected_regions,country):
    name = object_name[graph_object]
    filtered_happiness = happiness[(happiness['country'] == country) & (happiness['region'].isin(selected_regions))]
    line_fg = px.line(filtered_happiness, x = 'year', y = graph_object, title=f'{name} in {country}')
    return line_fg

# update change dựa trên sự thay đổi của 1 component khác - sử dụng Stats ( Ví dụ: sự thay đổi n_clicks của html.Button)
# State like Input, but the the output change can be control by click button
@app.callback(Output(avg_cal_info,'children'), 
              Input(avg_cal_update_button,'n_clicks'),
              State(object_ratio,'value'),
              State(region_checklist,'value'),
              State(country_dropdown,'value')) # số (input+state) và output của callback trùng với số input và output của hàm def
def update_avg_cal(button,graph_object, selected_regions,country):
    name = object_name[graph_object]
    filtered_happiness = happiness[(happiness['country'] == country) & (happiness['region'].isin(selected_regions))]
    avg_cal = round(filtered_happiness[graph_object].mean(),2)
    return f"The average of {name} is {avg_cal}"
    

app.run_server(mode='inline', host='localhost',port=8050, debug = True)

In [ ]:
# app.run_server('external')
# app.run_server(mode = 'inline')